# Agent Gateway Quickstart

Agent Gateway is a multi-agent framework that offers native support for Snowflake tools. 

The system can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Agent Gateway does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing the agent.

# Agent Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [ ]:
from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool, SQLTool
from snowflake.snowpark import Session
import os
from dotenv import load_dotenv

load_dotenv()

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

snowpark = Session.builder.configs(connection_parameters).create()

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [ ]:
search_config = {
    "service_name": "SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance.",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK", "RELATIVE_PATH"],
    "snowflake_connection": snowpark,
    "k": 10,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
    "max_results": 5,
}

## Python Tool Configuration

Configuring a Python Tool for the Agent Gateway requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [ ]:
import requests


def html_crawl(url):
    response = requests.get(url)
    return response.text


python_crawler_config = {
    "tool_description": "reads the html from a given URL or website",
    "output_description": "html of a webpage",
    "python_func": html_crawl,
}

# SQL Tool

The SQL Tool allows users to pre-define sql metrics and custom pipelines that the agent will be able to utilize to answer specialized questions.

In [ ]:
sql_query = """WITH CompanyMetrics AS (
    SELECT 
        LONGNAME,
        SECTOR,
        INDUSTRY,
        CURRENTPRICE,
        MARKETCAP,
        EBITDA,
        CASE 
            WHEN MARKETCAP > 0 AND EBITDA IS NOT NULL THEN (EBITDA * 100.0) / MARKETCAP
            ELSE NULL
        END AS EBITDA_Margin
    FROM CUBE_TESTING.PUBLIC.SP500
),
AverageMetrics AS (
    SELECT 
        AVG(EBITDA_Margin) AS Average_EBITDA_Margin
    FROM CompanyMetrics
),
NormalizedMetrics AS (
    SELECT 
        cm.LONGNAME,
        cm.SECTOR,
        cm.INDUSTRY,
        cm.CURRENTPRICE,
        cm.MARKETCAP,
        cm.EBITDA,
        cm.EBITDA_Margin,
        CASE 
            WHEN am.Average_EBITDA_Margin > 0 THEN cm.EBITDA_Margin / am.Average_EBITDA_Margin
            ELSE NULL
        END AS Normalized_EBITDA_Margin
    FROM CompanyMetrics cm
    CROSS JOIN AverageMetrics am
)
SELECT 
    LONGNAME,
    SECTOR,
    INDUSTRY,
    CURRENTPRICE,
    MARKETCAP,
    EBITDA,
    EBITDA_Margin,
    Normalized_EBITDA_Margin
FROM NormalizedMetrics;"""

sql_tool_config = {
    "name": "margin_eval",
    "connection": snowpark,
    "sql_query": sql_query,
    "tool_description": "Calculates the normalized EBITDA Margin as a % relative to the SP500 average",
    "output_description": "EBITDA Margin %",
}

## Agent Config

After the tools have been configured, initialize them and configure the agent.

In [ ]:
annual_reports = CortexSearchTool(**search_config)
sp500 = CortexAnalystTool(**analyst_config)
web_crawler = PythonTool(**python_crawler_config)
margin_eval = SQLTool(**sql_tool_config)


snowflake_tools = [annual_reports, sp500, web_crawler, margin_eval]
agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools, max_retries=3)

# Agent Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with the Agent Gateway. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [ ]:
agent("What is the market cap of Apple?")

In [ ]:
agent("What is MSFT's normalized EBITDA margin?")

In [ ]:
agent("Which company has the bigger EBITDA, Apple or MSFT?")

# Unstructured Data Questions

In [ ]:
agent("How many customers does Snowflake have?")

In [ ]:
agent(
    "On which platforms can I host Snowflake according to this documentation page https://docs.snowflake.com/en/user-guide/intro-cloud-platforms"
)

## Unstructured + Structured Data Questions

In [ ]:
agent(
    "What is the market cap of each of the cloud providers mentioned in Snowflake's annual report?"
)

install framework with requisite dependencies with `pip install orchestration-framework[trulens]` and initialize as follows:

# Agent Observability

In [ ]:
from agent_gateway import TruAgent
from trulens.connectors.snowflake import SnowflakeConnector

tru_conn = SnowflakeConnector(**connection_parameters)

agent = TruAgent(
    app_name="observable",
    app_version="v4",
    trulens_snowflake_connection=tru_conn,
    snowflake_connection=snowpark,
    tools=snowflake_tools,
    max_retries=3,
    evals=["fusion_groundedness", "answer_relevance"],
)

## Run the dashboard to view traces

In [ ]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession(connector=tru_conn)

run_dashboard(session, port=8084)